## Using a Jupyter notebook

This is a Jupyter notebook, that is, a collection of formatted text and live Python code.

Text and code are separated into cells. The active cell is highlighted with a border with a thicker left edge in blue or green. You can activate a different cell by clicking it.

To run all of the code within the active cell, just press Ctrl and Enter. `In [*]:` will appear at the top left corner while the code is running, and the `*` will turn into a number once it has finished. Any output from the code will then appear beneath the cell. *If you find at any point one of the text cells (e.g. this one) is no longer nicely formatted, but instead looks like code, just do the same: select it and press Ctrl and Enter to revert back.*

In the code, lines beginning with `#` are just comments. We use comment lines with arrows to indicate which parts of the code you are encouraged to modify:

```python
# Don't change this bit!
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Do edit this section!

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Don't edit this part either!
```

Code cells need to be run in the order that they appear in the notebook, as the later cells depend on the results of calculations from earlier cells.

---

# Total pH in a tris buffer solution

## Define the solution composition and conditions

The first thing we need to do is to define the conditions (i.e. temperature and pressure) that the solution is under, and the molality of each solute dissolved within it (i.e. its composition). The molalities are divided into fixed values for pH-conservative solutes, and 'total' values for species that will be allowed to equilibrate.

In [9]:
from numpy import log10, sqrt
import pytzer as pz
import pytzertools as pzt
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Set the temperature (in K) and pressure (in dbar)
tempK = 298.15
pres = 10.10325

# Define the solutes and their molalities
solutes = {
# First, the fixed-molality ions:
    'Na': 0.44516,
    'Ca': 0.01077,
    'K' : 0.01058,
    'Cl': 0.56912,
# Then, solutes involved in equilibria:
    't_HSO4': 0.02926,
    't_trisH': 0.08,
    't_Mg': 0.05518,
}

# Add some extra sodium sulfate
extra_Na2SO4 = 0.0
solutes['Na'] += extra_Na2SO4*2
solutes['t_HSO4'] += extra_Na2SO4

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Convert everything into arrays for Pytzer, and print out the concentrations to check they look sensible
solutearrays = pzt.solutes2arrays(solutes)
pzt.printmols(*solutearrays)

Fixed solute molalities:
0.44516 mol/kg-H2O = [Na]
0.01077 mol/kg-H2O = [Ca]
0.01058 mol/kg-H2O = [K]
0.56912 mol/kg-H2O = [Cl]

Equilibrating solute total molalities:
0.02926 mol/kg-H2O = [HSO4]
0.05518 mol/kg-H2O = [Mg]
0.08000 mol/kg-H2O = [trisH]


## Solve for equilibrium

Now we can solve for the solution's equilibrium speciation.

In [2]:
# Calculate and display equilibrium speciation
allmols, allions = pzt.solve(solutearrays, tempK, pres)
pzt.printmols(allions, allmols)

Solute molalities:
0.44516 mol/kg-H2O = [Na]
0.01077 mol/kg-H2O = [Ca]
0.01058 mol/kg-H2O = [K]
0.56912 mol/kg-H2O = [Cl]
0.00000 mol/kg-H2O = [HSO4]
0.02926 mol/kg-H2O = [SO4]
0.05518 mol/kg-H2O = [Mg]
0.00000 mol/kg-H2O = [MgOH]
0.04001 mol/kg-H2O = [trisH]
0.03999 mol/kg-H2O = [tris]
0.00000 mol/kg-H2O = [H]
0.00000 mol/kg-H2O = [OH]


In [3]:
# Extract H+ and HSO4- molalities from equilibrium results
mH = allmols[allions == 'H']
mHSO4 = allmols[allions == 'HSO4']

# Add together to get Total scale pH
pH_Total = -log10(mH + mHSO4)

# Display the result
print('Total scale pH = {:.3f}'.format(pH_Total[0]))

Total scale pH = 8.058


In [4]:
# Calculate derivative of pH w.r.t. each ln(K)
pHgrads = pzt.pHgrads(solutearrays, tempK, pres)

0.09850009430323325 = dpH/dK(HSO4)
-0.4341346144798308 = dpH/dK(trisH)
-8.279865681970477e-05 = dpH/dK(MgOH)
-0.00016112000622570122 = dpH/dK(H2O)


In [10]:
uncert_lnk = {}
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Define uncertainties
uncert_lnk['HSO4'] = 0.0484
uncert_lnk['trisH'] = 0.01 # PLACEHOLDER VALUE!!!
uncert_lnk['MgOH'] = 0.022
uncert_lnk['H2O'] = 0.023

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Propagate and print out uncertainties
uncert_pHT = {eq: uncert_lnk[eq]*pHgrads[eq] for eq in pHgrads}
total_uncert_pHT = sqrt(sum([uncert_pHT[eq]**2 for eq in uncert_pHT]))
for eq in pHgrads:
    print('{} = uncert. in pH due to K({})'.format(uncert_pHT[eq], eq))
print(total_uncert_pHT)

0.0047674045642764895 = uncert. in pH due to K(HSO4)
-0.004341346144798308 = uncert. in pH due to K(trisH)
-1.821570450033505e-06 = uncert. in pH due to K(MgOH)
-3.705760143191128e-06 = uncert. in pH due to K(H2O)
0.006447902734937662
